This notebook is to import, clean and combine Saudia's aircraft configuration data and export it into a dataset

In [1]:
# imports
import numpy as np
import pandas as pd
import re

In [2]:
a320 = pd.read_csv('Airbus A320-214.csv')
a321 = pd.read_csv('Airbus A321.csv')
a330 = pd.read_csv('Airbus A330-343.csv')
b772 = pd.read_csv('Boeing B777-268ER.csv')
b773 = pd.read_csv('Boeing B777-368ER.csv')
b787 = pd.read_csv('Boeing B787-9.csv')

In [3]:
# create a unified dataframe
df = pd.concat([a320,a321,a330,b772,b773,b787], sort=False).reset_index(drop=True)

In [4]:
# move 'has_wifi_connectivity' column values to 'has_wifi' and drop extras
df['saudia_first_suite_has_wi_fi'] = df.saudia_first_suite_has_wi_fi_connectivity
df['economy_class_has_wi_fi'][df['economy_class_has_wi_fi_connectivity'] == True] = True
df['business_class_has_wi_fi'][df['business_class_has_wi_fi_connectivity'] == True] = True
df['first_class_has_wi_fi'][df['first_class_has_wi_fi_connectivity'] == True] = True
df.drop(['saudia_first_suite_has_wi_fi_connectivity','economy_class_has_wi_fi_connectivity','business_class_has_wi_fi_connectivity','first_class_has_wi_fi_connectivity'], axis=1, inplace=True)

C:\Users\TQ\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\TQ\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\TQ\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [5]:
# fill boolean column NANs with false
bool_cols = df.columns[df.columns.str.contains('_has_')]
df[bool_cols] = df[bool_cols].fillna(False)

In [6]:
# fill other NANs with 'class not offered'
df.fillna('class not offered', inplace=True)

In [7]:
# extract int values from columns and fix 787 values
df.rename(columns={'length':'length_in_m'}, inplace=True)
df.length_in_m = df.length_in_m.apply(lambda x:re.search(r'([\d\.]+) m', x).group(1).replace('.', '')).astype('int64')
df.length_in_m.iloc[15] = df.length_in_m.iloc[15]*10

C:\Users\TQ\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
df.rename(columns={'wingspan':'wingspan_in_m'}, inplace=True)
df.wingspan_in_m = df.wingspan_in_m.apply(lambda x:re.search(r'([\d\.]+) m', x).group(1).replace('.', '')).astype('int64')
df.wingspan_in_m.iloc[15] = df.wingspan_in_m.iloc[15]*10

In [9]:
df.rename(columns={'range':'range_in_km'}, inplace=True)
df.range_in_km = df.range_in_km.apply(lambda x:re.search(r'([\d\.,-]+) km', x).group(1).replace(',', '').replace('.', ''))
b787_range = df.loc[15,'range_in_km'].split('-')
b787_range = int((int(b787_range[0]) + int(b787_range[1]))/len(b787_range))
df.loc[15,'range_in_km'] = b787_range
df.range_in_km = df.range_in_km.astype('int64')

In [10]:
df.rename(columns={'maximum_cruising_altitude':'maximum_cruising_altitude_in_ft'}, inplace=True)
df.maximum_cruising_altitude_in_ft = df.maximum_cruising_altitude_in_ft.apply(lambda x:re.search(r'([\d\.,-]+) ft', x).group(1).replace(',', '').replace('.', ''))
aceo_maxalt = df.loc[0,'maximum_cruising_altitude_in_ft'].split('-')
aceo_maxalt = int((int(aceo_maxalt[0]) + int(aceo_maxalt[1]))/len(aceo_maxalt))
df.loc[0,'maximum_cruising_altitude_in_ft'] = aceo_maxalt
df.maximum_cruising_altitude_in_ft = df.maximum_cruising_altitude_in_ft.astype('int64')

In [11]:
df.rename(columns={'cruising_speed':'cruising_speed_in_kmh'}, inplace=True)
df.cruising_speed_in_kmh = df.cruising_speed_in_kmh.apply(lambda x:re.search(r'([\d]+) km/h', x).group(1)).astype('int64')

In [12]:
pitch_width_cols = df.columns[df.columns.str.contains('seat_pitch') | df.columns.str.contains('seat_width')]

In [13]:
# fill in 772 1st class width pitch and abreast data from seatguru since data missing on saudia website
df.first_class_seat_width.iloc[8] = '21'
df.first_class_seat_pitch.iloc[8] = '79'
df.first_class_abreast.iloc[8] = '2-2-2'

In [14]:
for col in pitch_width_cols:
    df[col] = df[col].apply(lambda x:re.search(r'([\d\.]+) in', x).group(1) if 'in' in x else x)
    df.rename(columns={col:col+'_in_inches'}, inplace=True)

In [15]:
# export as csv dataset
df.to_csv('./datasets/saudia_aircraft_configs_data.csv', index=False)